In [1]:
import click
import pandas as pd
import os
import time

from binance.client import Client

In [2]:
api_key = 'XzkhEsU0HHARj880CT9Ck4tlsD4WZaQSbsbdAy7kXvMp3yOjzKdRbCZdGAeQ2YhA'

api_secret = 'xfg149pnCFuhWHOtgHOmgwCox4DkGRv66FI0kN5VNY3FLuhMhsfhJxJRMIGPeGiQ'

client = Client(api_key, api_secret)

In [3]:
coins = client.get_all_coins_info()

coins_dataframe = pd.DataFrame(columns=['coin','trading','isLegalMoney'])

for coin in coins:
    coins_dataframe = coins_dataframe.append({'coin': coin['coin'], 'trading': coin['trading'],'isLegalMoney': coin['isLegalMoney']}, ignore_index=True) 

coins_dataframe = coins_dataframe[coins_dataframe.trading > 0 ]
coins_dataframe = coins_dataframe[coins_dataframe.isLegalMoney == 0]

coins_dataframe['trade_fee'] = coins_dataframe.coin.apply(lambda x: client.get_trade_fee(symbol=x+'USDT'))

coins_dataframe.trade_fee = coins_dataframe.trade_fee.apply(lambda x: x if len(x)> 0 else None)

coins_dataframe = coins_dataframe[coins_dataframe.trade_fee.astype(str) != 'None']

coins_dataframe['trade_symbol'] = coins_dataframe.trade_fee.apply(lambda x: x[0]['symbol'])
coins_dataframe['maker_commission'] = coins_dataframe.trade_fee.apply(lambda x: x[0]['makerCommission'])
coins_dataframe['taker_commission'] = coins_dataframe.trade_fee.apply(lambda x: x[0]['takerCommission'])

In [49]:
#coins_dataframe.apply(lambda x: x.trade_symbol.split(x.coin)[-1], axis=1).unique()[0]

'USDT'

In [4]:
coins_dataframe.reset_index(inplace=True,drop=True)

coins_dataframe['profit'] = 0

coins_dataframe['gained'] = 0

coins_dataframe['times_it_sold'] = 0

coins_dataframe['coin_status'] = 'initialized'

In [9]:
prices = client.get_all_tickers()
for price in prices:
    if price['symbol'] in coins_dataframe.trade_symbol.values:
        index = coins_dataframe[coins_dataframe.trade_symbol == price['symbol']].index[0]
        
        ticker = client.get_ticker(symbol=price['symbol'])
        
        coins_dataframe.loc[index, 'price'] = price['price']
        coins_dataframe.loc[index, 'market'] = price['symbol'].split(coins_dataframe.loc[index].coin)[-1].strip()
        
        coins_dataframe.loc[index, 'price_change'] = ticker['priceChange']
        coins_dataframe.loc[index, 'price_change_percent']= ticker['priceChangePercent']
        coins_dataframe.loc[index, 'high_price'] = ticker['highPrice']
        coins_dataframe.loc[index, 'low_price'] = ticker['lowPrice']
        coins_dataframe.loc[index, 'coin_24h_vol'] = ticker['volume']
        coins_dataframe.loc[index, 'trade_symbol_24h_vol'] = ticker['quoteVolume']
        
        print(price)
        

{'symbol': 'BTCUSDT', 'price': '47235.16000000'}
{'symbol': 'ETHUSDT', 'price': '3320.01000000'}
{'symbol': 'BNBUSDT', 'price': '404.30000000'}
{'symbol': 'NEOUSDT', 'price': '48.42000000'}
{'symbol': 'LTCUSDT', 'price': '176.50000000'}


KeyboardInterrupt: 

In [21]:
client.get_ticker(symbol='ETHUSDT')

{'symbol': 'ETHUSDT',
 'priceChange': '-115.10000000',
 'priceChangePercent': '-3.305',
 'weightedAvgPrice': '3397.14438419',
 'prevClosePrice': '3483.00000000',
 'lastPrice': '3367.91000000',
 'lastQty': '0.18640000',
 'bidPrice': '3367.91000000',
 'bidQty': '30.26660000',
 'askPrice': '3367.92000000',
 'askQty': '0.70470000',
 'openPrice': '3483.01000000',
 'highPrice': '3489.99000000',
 'lowPrice': '3302.00000000',
 'volume': '310554.97760000',
 'quoteVolume': '1055000098.13577900',
 'openTime': 1631979609140,
 'closeTime': 1632066009140,
 'firstId': 604321924,
 'lastId': 605004288,
 'count': 682365}

In [24]:
products = client.get_products()


In [38]:
products_dict = {w['s'].replace('USDT',''):w['cs'] for w in products['data'] if 'USDT' in w['s']}
#[products_dict.update({}) ]

In [43]:
'ADADOWN' in products_dict.keys()

False

In [42]:
products_dict['']

KeyError: 'ADADOWN'

In [39]:
products_dict

{'BTC': 18820031,
 '1INCH': 180362122,
 'AAVE': 13181524,
 'ACM': 34698900,
 'ADA': 32038100544,
 'AION': 495965163,
 'AKRO': 3244483103,
 'ALGO': 5386243616,
 'ALICE': 23000000,
 'ALPACA': 141603872,
 'ALPHA': 406330126,
 'ANKR': 7662899378,
 'ANT': 0,
 'ARDR': 998999495,
 'ARPA': 1134168018,
 'AR': 33394701,
 'ASR': 2133249,
 'ATA': 172252000,
 'ATM': 2086980,
 'ATOM': 221601834,
 'AUDIO': 406077610,
 'AUD': None,
 'AUTO': 53003,
 'AVA': 51530587,
 'AVAX': 220286577,
 'AXS': 60907500,
 'BADGER': 10116031,
 'BAKE': 193529499,
 'BAL': 6943831,
 'BAND': 35191821,
 'BAT': 0,
 'BCH': 18849850,
 'BEAM': 97104120,
 'BEL': 48000000,
 'BLZ': 308118749,
 'BNT': 232079685,
 'BOND': 384149391,
 'BTCST': 11417109,
 'BTT': 659952625000,
 'BURGER': 17084466,
 'BRY': 12813558456,
 'BUSD': 12813558456,
 'BZRX': 305092355,
 'C98': 185000000,
 'CAKE': 222955519,
 'CELO': 306819314,
 'CELR': 5852080630,
 'CFX': 1001638647,
 'CHR': 429822300,
 'CHZ': 5897640034,
 'CKB': 27888057738,
 'CLV': 128777778,
 '

In [18]:
dir(client)

['AGG_BEST_MATCH',
 'AGG_BUYER_MAKES',
 'AGG_FIRST_TRADE_ID',
 'AGG_ID',
 'AGG_LAST_TRADE_ID',
 'AGG_PRICE',
 'AGG_QUANTITY',
 'AGG_TIME',
 'API_KEY',
 'API_SECRET',
 'API_TESTNET_URL',
 'API_URL',
 'COIN_FUTURE_TO_SPOT',
 'FIAT_TO_MINING',
 'FIAT_TO_SPOT',
 'FIAT_TO_USDT_FUTURE',
 'FUTURES_API_VERSION',
 'FUTURES_API_VERSION2',
 'FUTURES_COIN_DATA_TESTNET_URL',
 'FUTURES_COIN_DATA_URL',
 'FUTURES_COIN_TESTNET_URL',
 'FUTURES_COIN_URL',
 'FUTURES_DATA_TESTNET_URL',
 'FUTURES_DATA_URL',
 'FUTURES_TESTNET_URL',
 'FUTURES_URL',
 'FUTURE_ORDER_TYPE_LIMIT',
 'FUTURE_ORDER_TYPE_LIMIT_MAKER',
 'FUTURE_ORDER_TYPE_MARKET',
 'FUTURE_ORDER_TYPE_STOP',
 'FUTURE_ORDER_TYPE_STOP_MARKET',
 'FUTURE_ORDER_TYPE_TAKE_PROFIT',
 'FUTURE_ORDER_TYPE_TAKE_PROFIT_MARKET',
 'KLINE_INTERVAL_12HOUR',
 'KLINE_INTERVAL_15MINUTE',
 'KLINE_INTERVAL_1DAY',
 'KLINE_INTERVAL_1HOUR',
 'KLINE_INTERVAL_1MINUTE',
 'KLINE_INTERVAL_1MONTH',
 'KLINE_INTERVAL_1WEEK',
 'KLINE_INTERVAL_2HOUR',
 'KLINE_INTERVAL_30MINUTE',
 'KLINE_

In [10]:
ticker

{'symbol': 'LTCUSDT',
 'priceChange': '-5.90000000',
 'priceChangePercent': '-3.236',
 'weightedAvgPrice': '180.46223605',
 'prevClosePrice': '182.30000000',
 'lastPrice': '176.40000000',
 'lastQty': '2.18800000',
 'bidPrice': '176.40000000',
 'bidQty': '68.69400000',
 'askPrice': '176.50000000',
 'askQty': '630.21500000',
 'openPrice': '182.30000000',
 'highPrice': '185.40000000',
 'lowPrice': '175.70000000',
 'volume': '320774.09500000',
 'quoteVolume': '57887610.45110000',
 'openTime': 1631966752668,
 'closeTime': 1632053152668,
 'firstId': 200196119,
 'lastId': 200288616,
 'count': 92498}

In [44]:
coins_dataframe.reset_index(drop=True,inplace=True)

for coin in coins_dataframe.coin.values:
    index = coins_dataframe[coins_dataframe.coin == coin].index[0]
    
    print(coins_dataframe.loc[index].coin)

STPT
RAY
NEAR
AUDIO
HNT
ADADOWN
SUSD
FARM
AION
DGB
ZRX
WING
WNXM
BCH
JST
ADAUP
HOT
AR
IRIS
RAMP
FLM
GNO
VITE
CFX
SFP
DIA
ARDR
BEL
JUV
ACM
MINA
VTHO
REN
REP
ADA
ELF
REQ
STORJ
BZRX
EOSDOWN
PAXG
CHR
BCHDOWN
WAVES
CHZ
XRP
SAND
OCEAN
FOR
UMA
TUSD
TKO
ENJ
UNIDOWN
YFII
GRT
HARD
TFUEL
TROY
XLMUP
UNI
BTCDOWN
TLM
CKB
WRX
XTZ
LUNA
ETHDOWN
EOS
RIF
SKL
GTC
SXPDOWN
BURGER
GTO
WTC
XVG
EPS
DNT
CLV
FLOW
XTZDOWN
XVS
SLP
NBS
DOT
1INCH
MITH
DEGO
UNFI
FTM
ERN
FTT
PHA
RLC
TRXDOWN
ATOM
XRPUP
QUICK
BLZ
MBL
BNBUP
SNX
LTCDOWN
FUN
COS
SUSHIUP
ROSE
SOL
TRXUP
ETC
BNB
CELR
OGN
ETH
NEO
TOMO
CELO
KLAY
BADGER
GXS
TRB
BNT
UTK
TRU
DREP
TRX
MDT
MDX
XRPDOWN
NULS
AUTO
EGLD
PUNDIX
CRV
BAKE
ANT
NU
LINKUP
SRM
TORN
QNT
ALICE
OG
MFT
OM
BUSD
CTK
ARPA
DOTDOWN
MATIC
IOTX
SHIB
TVK
DOCK
STX
PNT
DENT
MBOX
IOST
CAKE
ETHUP
OMG
BAND
SUN
BTC
TWT
NKN
RSR
IOTA
CVC
REEF
BTG
MIR
KEY
BTS
BTT
ONE
LINKDOWN
ONG
ANKR
SUSHI
ALGO
SC
ONT
ASR
FIRO
MANA
DOTUP
ATA
NMR
MKR
DODO
LIT
ICP
ZEC
ATM
ICX
ZEN
DOGE
DUSK
ALPHA
SXP
HBAR
RVN
MLN
NANO
CTSI
KAVA
C98


In [40]:
coins_dataframe

,coin,trading,isLegalMoney,trade_fee,trade_symbol,maker_commission,taker_commission
0,STPT,True,False,"[{'symbol': 'STPTUSDT', 'makerCommission': '0....",STPTUSDT,0.001,0.001
5,RAY,True,False,"[{'symbol': 'RAYUSDT', 'makerCommission': '0.0...",RAYUSDT,0.001,0.001
6,NEAR,True,False,"[{'symbol': 'NEARUSDT', 'makerCommission': '0....",NEARUSDT,0.001,0.001
7,AUDIO,True,False,"[{'symbol': 'AUDIOUSDT', 'makerCommission': '0...",AUDIOUSDT,0.001,0.001
8,HNT,True,False,"[{'symbol': 'HNTUSDT', 'makerCommission': '0.0...",HNTUSDT,0.001,0.001
...,...,...,...,...,...,...,...
501,BAT,True,False,"[{'symbol': 'BATUSDT', 'makerCommission': '0.0...",BATUSDT,0.001,0.001
503,AKRO,True,False,"[{'symbol': 'AKROUSDT', 'makerCommission': '0....",AKROUSDT,0.001,0.001
505,XMR,True,False,"[{'symbol': 'XMRUSDT', 'makerCommission': '0.0...",XMRUSDT,0.001,0.001
506,1INCHDOWN,True,False,"[{'symbol': '1INCHDOWNUSDT', 'makerCommission'...",1INCHDOWNUSDT,0.001,0.001


In [ ]:
# client.get_all_tickers()
# client.get_historical_trades()
# # client.get_historical_trades(symbol='ETHUSDT')
# # client.get_ticker(symbol='ETHUSDT')
# # client.transfer_history(symbol='ETHUSDT')
# # client.transfer_history(asset='ETHUSDT')
# # client.get_recent_trades(asset='ETHUSDT')
# client.get_recent_trades(symbol='ETHUSDT')
# client.get_orderbook_ticker(symbol='ETHUSDT')
# client.get_margin_symbol(symbol='ETHUSDT')
# client.get_klines(symbol='ETHUSDT')
# client.get_historical_trades(symbol='ETHUSDT')
# client.get_universal_transfer_history(symbol='ETHUSDT')
# client.get_orderbook_ticker(symbol='ETHUSDT')
# client.get_margin_symbol(symbol='ETHUSDT')
# client.get_historical_trades(symbol='ETHUSDT')

In [31]:
client.get_all_tickers()

[{'symbol': 'ETHBTC', 'price': '0.07139600'},
 {'symbol': 'LTCBTC', 'price': '0.00375300'},
 {'symbol': 'BNBBTC', 'price': '0.00852600'},
 {'symbol': 'NEOBTC', 'price': '0.00102700'},
 {'symbol': 'QTUMETH', 'price': '0.00362600'},
 {'symbol': 'EOSETH', 'price': '0.00152700'},
 {'symbol': 'SNTETH', 'price': '0.00002715'},
 {'symbol': 'BNTETH', 'price': '0.00118700'},
 {'symbol': 'BCCBTC', 'price': '0.07908100'},
 {'symbol': 'GASBTC', 'price': '0.00020280'},
 {'symbol': 'BNBETH', 'price': '0.11950000'},
 {'symbol': 'BTCUSDT', 'price': '48175.34000000'},
 {'symbol': 'ETHUSDT', 'price': '3439.55000000'},
 {'symbol': 'HSRBTC', 'price': '0.00041400'},
 {'symbol': 'OAXETH', 'price': '0.00017780'},
 {'symbol': 'DNTETH', 'price': '0.00002801'},
 {'symbol': 'MCOETH', 'price': '0.00577200'},
 {'symbol': 'ICNETH', 'price': '0.00166300'},
 {'symbol': 'MCOBTC', 'price': '0.00021140'},
 {'symbol': 'WTCBTC', 'price': '0.00002440'},
 {'symbol': 'WTCETH', 'price': '0.00023700'},
 {'symbol': 'LRCBTC', 'p

In [32]:
client.get_ticker(symbol='ETHUSDT')

{'symbol': 'ETHUSDT',
 'priceChange': '-75.88000000',
 'priceChangePercent': '-2.157',
 'weightedAvgPrice': '3448.04840791',
 'prevClosePrice': '3517.27000000',
 'lastPrice': '3441.39000000',
 'lastQty': '0.00580000',
 'bidPrice': '3441.39000000',
 'bidQty': '5.00420000',
 'askPrice': '3441.40000000',
 'askQty': '0.13570000',
 'openPrice': '3517.27000000',
 'highPrice': '3541.96000000',
 'lowPrice': '3373.72000000',
 'volume': '266149.47290000',
 'quoteVolume': '917696266.29980500',
 'openTime': 1631953815779,
 'closeTime': 1632040215779,
 'firstId': 604121789,
 'lastId': 604727275,
 'count': 605487}